## 1. Why decorators exist

In Python, **functions are first-class objects**:


When we say **“functions are first-class objects”**, it means:

> **Functions are treated like any other value in the language**
> (just like integers, strings, or objects)

In Python, this has very concrete and practical consequences.

---

#### 1.1. What “first-class” means (precisely)

An entity is *first-class* if it can:

- ✅ be assigned to a variable
- ✅ be passed as an argument to a function
- ✅ be returned from a function
- ✅ be stored in data structures

Python functions satisfy **all four**.

---

#### 1.2. Functions can be assigned to variables

```python
def greet():
    print("Hello")

f = greet      # no parentheses
f()
```

Key point:

* `greet` is the **function object**
* `greet()` is the **result of calling it**

---

#### 1.3. Functions can be passed as arguments

```python
def run(func):
    func()

def say_hi():
    print("Hi")

run(say_hi)
```

This works because `say_hi` is just a value.

---

#### 1.4. Functions can be returned from other functions

```python
def make_adder(x):
    def add(y):
        return x + y
    return add

add5 = make_adder(5)
print(add5(3))
```

Output:

```
8
```





**1️⃣ Defining `make_adder`**

```python
def make_adder(x):
    ...
```

At this point:

* `make_adder` is **just a function object**
* Nothing has executed yet
* `x` has **no value yet**

---

**2️⃣ Calling `make_adder(5)`**

```python
add5 = make_adder(5)
```

Now the function **executes**.

* `x = 5`
* Python enters `make_adder`

Inside it, Python **defines another function**:

```python
def add(y):
    return x + y
```

Important points:

* `add` **does not execute**
* `add` **captures** the variable `x`
* `x` lives in the **enclosing scope** of `add`

Then:

```python
return add
```

So:

* `make_adder(5)` returns the function `add`
* That function remembers `x = 5`

Result:

```python
add5  # is now a function
```

But internally:

```text
add5(y) = 5 + y
```

---

**3️⃣ Calling `add5(3)`**

```python
print(add5(3))
```

Now the inner function runs.

* `y = 3` (argument of `add`)
* `x = 5` (remembered from when `make_adder` ran)

The return statement executes:

```python
return x + y
```

So:

```text
5 + 3 = 8
```

---


| Variable | Value | Where it comes from       |
| -------- | ----- | ------------------------- |
| `x`      | `5`   | Passed to `make_adder(5)` |
| `y`      | `3`   | Passed to `add5(3)`       |
| Result   | `8`   | `x + y`                   |

---

**Why this works**

This is called a **closure**.

* `add` **closes over** `x`
* Even though `make_adder` has finished execution
* `x` is **still alive** because `add` uses it

You can inspect it:

```python
print(add5.__closure__[0].cell_contents)
```

Output:

```
5
```

---

**Mental model**

Think of it like this:

```text
make_adder(5)
    ↓
returns a function:
    add(y) = 5 + y
```

Then:

```text
add5(3)
→ 5 + 3
→ 8
```

---

**One-sentence takeaway**

* **`x = 5`** is fixed when `make_adder` is called
* **`y = 3`** is provided later when the returned function is called
* The inner function remembers `x` via a **closure**

---

#### 1.5. Functions can be stored in data structures

```python
def add(a, b):
    return a + b

def sub(a, b):
    return a - b

ops = {
    "add": add,
    "sub": sub
}

print(ops["add"](2, 1))
```

---

#### 1.6. Functions have identity and attributes

Functions are objects:

```python
def f():
    pass

print(type(f))
print(f.__name__)
print(f.__doc__)
```

Output:

```
<class 'function'>
f
None
```

They live in memory and have metadata.

---

#### 1.7. Why this matters (the big picture)

Because functions are first-class:

* decorators are possible
* callbacks are possible
* event systems work
* web frameworks (FastAPI, Flask) work
* functional programming patterns work

Example (decorator connection):

```python
def decorator(func):
    def wrapper():
        print("Before")
        func()
        print("After")
    return wrapper
```

This only works because `func` is a value.

---

#### 1.8. Contrast with languages where functions are NOT first-class

In older languages (or limited ones):

❌ functions cannot be passed as values
❌ callbacks require special syntax
❌ decorators are impossible

Python does not have these limitations.

---

#### 1.9. Very short mental model

Think of a function as:

> “A value that happens to be callable”

Just like:

```python
x = 5
x()

❌ TypeError
```

but:

```python
f = greet
f()

✅
```

---

## 2. A simple decorator (manual version)


A **decorator** lets you:

* Wrap a function
* Add behavior **before and/or after** it
* Without modifying the original function code

Typical uses:

* Logging
* Timing
* Authentication / authorization
* Caching
* Input validation

Goal: add behavior around a function.

```python
def my_decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper
```

Use it manually:

```python
def say_hello():
    print("Hello!")

say_hello = my_decorator(say_hello)
say_hello()
```

Output:

```
Before function call
Hello!
After function call
```

---

## 3. The `@decorator` syntax (clean way)

Python provides syntax sugar:

```python
def my_decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

This is **exactly the same** as:

```python
say_hello = my_decorator(say_hello)
```

---


When you see:

```python
@decorator
def f():
    pass
```

Think:

```python
f = decorator(f)
```

---

## 4. Decorators and Arguments
## 4.1 Case A — Function takes arguments (still ONE wrapper)

### Code

```python
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before")
        result = func(*args, **kwargs)
        print("After")
        return result
    return wrapper
```

### Usage

```python
@my_decorator
def add(a, b):
    return a + b

add(2, 3)
```

---

### What Python rewrites

```python
def add(a, b):
    return a + b

add = my_decorator(add)
print(add(2, 3))
```

---

### Why only one wrapper?

* `my_decorator` receives **the function**
* `*args, **kwargs` belong to **wrapper**
* wrapper runs **at call time**
* decorator still has **no configuration**

So we still need **only one wrapper**.

---

## 4.2 Case B — Decorator takes arguments (TWO wrappers)

### Code

```python
def my_decorator_with_arg(message):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(message)
            return func(*args, **kwargs)
        return wrapper
    return decorator
```

### Usage

```python
@my_decorator_with_arg("Before")
def add(a, b):
    return a + b

print(add(2, 3))    
```

---

### What Python rewrites

```python
def add(a, b):
    return a + b

add = my_decorator_with_arg("Before")(add)
```

---

## Side-by-side comparison (this is the mental model)

| Aspect               | Function args       | Decorator args           |
| -------------------- | ------------------- | ------------------------ |
| `*args, **kwargs`    | belong to `wrapper` | belong to outer function |
| When evaluated       | call time           | decoration time          |
| Extra wrapper needed | ❌ No                | ✅ Yes                    |
| Example              | `add(a, b)`         | `@decorator("msg")`      |

---

## Example FastAPI @app.get(...)

You want this to work:

```python
app = FastAPI()

@app.get("/ping")
def ping():
    print("pong")
```

So `get` must:

1. receive a **path** (like `"/ping"`)
2. receive a **function**
3. return a **wrapper**
4. register the function internally
5. execute it later when a request comes in

---

## Minimal implementation

```python
class FastAPI:
    def __init__(self) -> None:
        self.routes = {}

    def get(self, path: str):
        def register_handler(handler: callable): # decorator
            # register route
            self.routes[("GET", path)] = handler

            def route_handler(): # wrapper
                print("processing network request")
                return handler()

            return route_handler

        return register_handler
```

---

## How this is used

```python
app = FastAPI()

@app.get("/ping")
def ping():
    print("pong")
```

Equivalent to:

```python
def ping():
    print("pong")

ping = app.get("/ping")(ping)
```

---

## What each name means (important)

### `get(self, path: str)`

Represents:

> “I am defining a GET route for this path”

---

### `register_handler(handler)`

This is the **decorator function**.

* `handler` is the user’s function (`ping`)
* This runs **at decoration time**

---

### `route_handler()`

This is the **wrapper**.

* It simulates:

  * request handling
  * middleware
  * logging
  * timing
* This runs **at request time**

Good name because:

* it handles a route
* it wraps the real handler

---

## Simulating a request

Add this:

```python
def simulate_request(self, method: str, path: str):
    handler = self.routes.get((method, path))
    if not handler:
        print("404 Not Found")
        return
    handler()
```

Now:

```python
app.simulate_request("GET", "/ping")
```

Output:

```
processing network request
pong
```

---

## Mental model (this is the key insight)

```text
@app.get("/ping")
↓
get("/ping")
↓
register_handler(ping)
↓
store ("GET", "/ping") → ping
↓
return wrapper
```

Later:

```text
HTTP GET /ping
↓
lookup route
↓
call wrapper
↓
call ping
```

---


## 1️⃣ What `self.routes = {}` really is

```python
self.routes = {}
```

This is just a **registry**.

Conceptually, it is:

> “A lookup table that maps an HTTP request → a Python function”

Think of it as a phone book.

---

## 2️⃣ What this line does

```python
self.routes[("GET", path)] = handler
```

It stores:

```text
(key)                 → (value)
("GET", "/ping")      → ping
```

Nothing is executed here.
No network.
No request.

It’s **just memory**.

---

## 3️⃣ When does this line run?

It runs at **decoration time**, not at request time.

Example:

```python
@app.get("/ping")
def ping():
    print("pong")
```

Python rewrites this as:

```python
ping = app.get("/ping")(ping)
```

So this happens **immediately when the module is loaded**.

---

## 4️⃣ Timeline (very important)

### Step A — app creation

```python
app = FastAPI()
```

Memory:

```python
app.routes == {}
```

---

### Step B — decorator executes

```python
@app.get("/ping")
def ping():
    print("pong")
```

This line runs:

```python
self.routes[("GET", "/ping")] = ping
```

Memory now:

```python
{
    ("GET", "/ping"): <function ping>
}
```

Still:

* no request
* no execution

---

### Step C — later, a request arrives

Conceptually:

```text
HTTP request: GET /ping
```

The framework does:

```python
handler = self.routes[("GET", "/ping")]
handler()
```

Which calls:

```python
ping()
```

---

## 5️⃣ Why the key is a tuple

```python
("GET", path)
```

Because:

```text
GET /users
POST /users
```

are **different routes**.

So the tuple uniquely identifies:

```text
(method, path)
```

---

## 6️⃣ This is exactly how routers work (conceptually)

Almost every web framework does something equivalent to:

```python
routes[(method, path)] = function
```

Then later:

```python
function = routes[(method, path)]
function()
```

---

## 7️⃣ What does *not* happen here (important)

❌ No HTTP server
❌ No sockets
❌ No JSON
❌ No async
❌ No threading

Just **mapping keys to callables**.

---


## One-sentence takeaway (lock this in)

> `self.routes` is just a **dictionary that remembers which function should be called for which HTTP method and path**, and decorators fill this dictionary at import time.




## 5. Preserving function metadata (`functools.wraps`)

Without this, the function name and docstring are lost.

```python
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper
```

Now:

```python
print(add.__name__)
```

Correctly prints:

```
add
```

---

---

## 6. Real-world example: timing a function

```python
import time
from functools import wraps

def timing(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Execution time: {end - start:.4f}s")
        return result
    return wrapper
```

Usage:

```python
@timing
def slow_function():
    time.sleep(1)

slow_function()
```

---